In [1]:
import sys
sys.path

['/users/mbarth/sirius/sirius/development',
 '/users/mbarth/miniconda3/envs/casa_env/lib/python37.zip',
 '/users/mbarth/miniconda3/envs/casa_env/lib/python3.7',
 '/users/mbarth/miniconda3/envs/casa_env/lib/python3.7/lib-dynload',
 '',
 '/users/mbarth/miniconda3/envs/casa_env/lib/python3.7/site-packages',
 '/users/mbarth/sirius',
 '/users/mbarth/miniconda3/envs/casa_env/lib/python3.7/site-packages/IPython/extensions',
 '/users/mbarth/.ipython']

In [4]:
import numpy as np
c = 299792458
#from direction_rotate import _calc_rotation_mats
import matplotlib.pyplot as plt


import xarray as xr
import dask.array as da
import cngi.conversion.convert_ms as convert_ms
import cngi.conversion.convert_image as convert_image
from cngi.dio import read_vis
import numpy as np
from astropy.coordinates import SkyCoord
from sirius import calc_vis
from astropy.wcs import WCS
import time
deg_to_rad = np.pi/180
from sirius import make_ant_sky_jones, simulation
from sirius._sirius_utils._ant_jones_term import _compute_rot_coords
from sirius._sirius_utils._calc_parallactic_angles import _calc_parallactic_angles, _find_optimal_set_angle

from astropy.time import Time
from astropy.coordinates import (EarthLocation, SkyCoord,
                                 AltAz, CIRS)
import astropy.units as u

import pkg_resources

ImportError: cannot import name 'calc_vis' from 'sirius' (unknown location)

In [5]:
tel_dir = pkg_resources.resource_filename('casadata', 'data/telescope_layout/data/vla.d.tel.zarr')
#tel_xds = xr.open_zarr(tel_dir)


mxds = read_vis('point_source_sim_vis/mosaic_pleiades_sim_dovp_True.vis.zarr')


mxds.attrs['xds0'] = mxds.attrs['xds0']#.isel(time=slice(80,81))
print(mxds.xds0.time)
#(_)_s dimension can be singleton (value is reused)

<xarray.DataArray 'time' (time: 350)>
array(['2019-10-04T05:48:39.760840448', '2019-10-04T05:54:39.760840448',
       '2019-10-04T06:00:39.760840448', ..., '2019-10-05T16:30:39.760840448',
       '2019-10-05T16:36:39.760840448', '2019-10-05T16:42:39.760840448'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2019-10-04T05:48:39.760840448 ... 2019-10-...


In [6]:
tel_dir

'/users/mbarth/miniconda3/envs/casa_env/lib/python3.7/site-packages/casadata/data/telescope_layout/data/vla.d.tel.zarr'

### Set up parameters for UVW calculation

In [8]:
ant_pos = mxds.ANTENNA.POSITION.values # [n_ant x 3]
time_str = mxds.xds0.time.values        # [n_time]
site = 'VLA'

phase_center_ra_dec = mxds.FIELD.PHASE_DIR.sel(field_id=mxds.xds0.FIELD_ID.isel(baseline=0).values).values[:,0,:] #n_time x 2
#print('phase_center_ra_dec',phase_center_ra_dec)
#print('field ids',mxds.xds0.FIELD_ID.isel(baseline=0).values)

ant1 = mxds.xds0.ANTENNA1.values
ant2 = mxds.xds0.ANTENNA2.values

In [9]:
uvw_parms = {}
uvw_parms['calc_method'] = 'astropy'
uvw_parms['site'] = 'vla'
uvw_parms['auto_corr'] = False

uvw = mxds.xds0.UVW.values
print(uvw.shape)

(350, 351, 3)


In [10]:
airy_disk_parms =  {'pb_func':'casa_airy','dish_diameter':24.5,'blockage_diameter':0.0}
beam_models = [airy_disk_parms]
n_ant = len(ant_pos)
beam_model_map = np.zeros(n_ant,dtype=int)

beam_parms = {}
beam_parms['pa_radius'] = 0.2
beam_parms['mueller_selection'] = np.array([0,15])

freq_chan = mxds.xds0.chan.values
pol = mxds.xds0.pol.values

### Setup Sources

In [11]:
#pointing_ra_dec:  [n_time, n_ant, 2]          (singleton: n_time, n_ant)
pointing_ra_dec = None #np.zeros((1, 1, 2)) #Singleton

#point_source_ra_dec:  [n_time, n_point_sources, 2]          (singleton: n_time)
ra = ['03h47m29.1s','03h49m09.7s','03h46m19.6s','  03h45m49.6s', '03h44m52.5s']
dec = ['+24d06m18.5s','+24d03m12.3s','+23d56m54.1s','+24d22m03.9s','+24d06m48.0s']
point_source_skycoord = SkyCoord(ra=ra,dec=dec,frame='fk5') #sim

'''
03h47m29.1s +24d06m18.5s
03h49m09.7s +24d03m12.3s
03h46m19.6s +23d56m54.1s
03h45m49.6s +24d22m03.9s
03h44m52.5s +24d06m48.0s
'''
point_source_ra_dec = np.array([point_source_skycoord.ra.rad,point_source_skycoord.dec.rad]).T[None,:,:]

In [12]:
# NB n_pol is no longer singleton
#point_source_flux: [n_time, n_chan, n_pol, n_point_sources] (singleton: n_time, n_chan)
point_source_flux = np.array([2.17, 0, 0, 2.17])[None,None,:,None] #has to be in instrument polarization: [RR,RL,LR,LL] or [XX,XY,YX,YY]. All 4 values are needed.
point_source_flux = np.tile(point_source_flux,(1,1,1,5))

pb_limit = 0.0045

In [17]:
vis_data, uvw = simulation.simulation(point_source_flux, point_source_ra_dec, pointing_ra_dec, phase_center_ra_dec, beam_parms,beam_models,beam_model_map,uvw_parms, ant_pos, time_str, freq_chan, pol, ant1, ant2, pb_limit, uvw) #None can be changed to uvw

TypeError: simulation() takes 4 positional arguments but 16 were given

In [ ]:
casa_vis_data = mxds.xds0.DATA.data.compute()

In [ ]:
dif = np.ravel((vis_data[:,:,:,0]-casa_vis_data[:,:,:,0])) #/np.abs(casa_vis_data[:,:,:,0])

plt.figure()
plt.plot(np.abs(dif))
plt.xlabel('Vis Number (ravel)')
plt.ylabel('Abs Dif')
plt.show()

plt.figure()
plt.plot(np.imag(dif))
plt.xlabel('Vis Number (ravel)')
plt.ylabel('Imag Dif')
plt.show()

plt.figure()
plt.plot(np.real(dif))
plt.xlabel('Vis Number (ravel)')
plt.ylabel('Real Dif')
plt.show()

plt.figure()
plt.plot(np.abs(np.ravel(vis_data[:,:,:,0])),label='Sirius')
plt.plot(np.abs(np.ravel(casa_vis_data[:,:,:,0])),label='CASA')
plt.legend()
plt.xlabel('Vis Number (ravel)')
plt.ylabel('abs Vis')
plt.show()

### Imaging

In [ ]:
from ngcasa.imaging import make_imaging_weight
from ngcasa.imaging import make_image
from ngcasa.imaging import make_pb
from ngcasa.imaging import make_gridding_convolution_function
from cngi.dio import write_image
import dask
import dask.array as da
from ngcasa.imaging import direction_rotate
from cngi.dio import write_image
from ngcasa.imaging import make_image_with_gcf
from ngcasa.imaging import make_mosaic_pb

rad_to_deg =  180/np.pi
deg_to_rad = np.pi/180
arcsec_to_deg = 1/3600
arcsec_to_rad = np.pi/(180*3600)

In [ ]:
#mxds = apply_flags(mxds, 'xds0', flags='FLAG')
mxds.attrs['xds1'] = mxds.attrs['xds0'].isel(pol=slice(0,1))
mxds.attrs['xds1']['DATA']
xr.DataArray(da.from_array(vis_data[:,:,:,0][:,:,:,None],chunks=mxds.attrs['xds1']['DATA'].chunks),coords=mxds.attrs['xds1']['DATA'].coords)

In [ ]:
grid_parms = {}
grid_parms['chan_mode'] = 'cube'
grid_parms['image_size'] = [1600,800]
grid_parms['cell_size'] = [5,5]

point_source_skycoord = SkyCoord(ra='03h47m29.1s',dec='+24d06m18.5s',frame='fk5')
grid_parms['phase_center'] = np.array([point_source_skycoord.ra.rad,point_source_skycoord.dec.rad])

In [ ]:
sel_parms = {}
sel_parms['xds'] = 'xds1' #KEEP USING XDS1 THROUGHOUT
sel_parms['data_group_in_id'] = 0 #CHANGE TO DATA GROUP 2 AFTER ROTATION

rotation_parms = {}
rotation_parms['new_phase_center'] = grid_parms['phase_center']
rotation_parms['common_tangent_reprojection'] = True
rotation_parms['single_precision'] = False

mxds = direction_rotate(mxds, rotation_parms, sel_parms)

In [ ]:
imaging_weights_parms = {}
imaging_weights_parms['weighting'] = 'natural'

sel_parms = {}
sel_parms['xds'] = 'xds1'
sel_parms['data_group_in_id'] = 2

mxds = make_imaging_weight(mxds, imaging_weights_parms, grid_parms, sel_parms)

In [ ]:
vis_sel_parms = {}
vis_sel_parms['xds'] = 'xds1'
vis_sel_parms['data_group_in_id'] = 2

img_sel_parms = {}
img_sel_parms['data_group_out_id'] = 0

In [ ]:
write_imagegrid_parms = {}
grid_parms['chan_mode'] = 'cube'
grid_parms['image_size'] = [1600,800]
grid_parms['cell_size'] = [5,5]

point_source_skycoord = SkyCoord(ra='03h47m29.1s',dec='+24d06m18.5s',frame='fk5')
grid_parms['phase_center'] = np.array([point_source_skycoord.ra.rad,point_source_skycoord.dec.rad])
gcf_parms = {}
gcf_parms['function'] = 'alma_airy'
gcf_parms['list_dish_diameters'] = np.array([24.5])
gcf_parms['list_blockage_diameters'] = np.array([0])
#gcf_parms['max_support'] = [24, 24]
#gcf_parms['oversampling'] = [10, 10]
gcf_parms['max_support'] = [48, 48]
gcf_parms['oversampling'] = [10, 10]
unique_ant_indx = mxds.ANTENNA.DISH_DIAMETER.values
unique_ant_indx[unique_ant_indx == 25.0] = 0
mxds.ANTENNA.DISH_DIAMETER.values
gcf_parms['unique_ant_indx'] = unique_ant_indx.astype(int)
gcf_parms['phase_center'] = grid_parms['phase_center']
sel_parms = {}
sel_parms['xds'] = 'xds1'
sel_parms['data_group_in_id'] = 2
gcf_xds = make_gridding_convolution_function(mxds, gcf_parms, grid_parms, sel_parms)
dask.compute(gcf_xds)

In [ ]:
img_xds = xr.Dataset() #empty dataset
vis_sel_parms = {}
vis_sel_parms['xds'] = 'xds1'
vis_sel_parms['data_group_in_id'] = 2

img_xds = make_mosaic_pb(mxds,gcf_xds,img_xds,vis_sel_parms,img_sel_parms,grid_parms)

In [ ]:
vis_select_parms = {}
vis_select_parms['xds'] = 'xds1'
vis_select_parms['data_group_in_id'] = 2

img_select_parms = {}
img_select_parms['data_group_in_id'] = 0
img_select_parms['data_group_out_id'] = 0

norm_parms = {}
norm_parms['norm_type'] = 'flat_sky'

img_xds = make_image_with_gcf(mxds,gcf_xds, img_xds, grid_parms, norm_parms, vis_select_parms, img_select_parms)

In [ ]:
chan = 1
ngcasa_image_name = 'IMAGE'
pb_limit = 0.2
#extent = extent=(np.min(casa_img_xds.m),np.max(casa_img_xds.m),np.min(casa_img_xds.l),np.max(casa_img_xds.l))

mosaic_pb = img_xds.PB.isel(chan=chan)
mosaic_img = img_xds[ngcasa_image_name].isel(chan=chan)
mosaic_img = mosaic_img.where(mosaic_pb > np.max(mosaic_pb)*pb_limit,other=np.nan)

plt.figure()
#plt.imshow(img_xds.IMAGE.isel(chan=chan,time=0,pol=0))
plt.imshow(mosaic_img[:, :, 0, 0])
